In [717]:
from tools import analysistools as atools
import nanoparticle
from nanoparticle import Ligand, NanoParticle
import math
from membranesimulation import MembraneSimulation
import numpy as np
from tools import icosatiler
from tools import vectools
from tools import misctools
import k3d
import os
import random
import pickle
from numpy import pi, cos, sin, arccos, arange

In [541]:
mag = lambda v : np.sqrt(np.sum([i*i for i in v]))

In [542]:
def goldenSpiral(n,rad):
    num_pts = 60
    indices = arange(0, n, dtype=float) + 0.5
    phi = arccos(1 - 2*indices/n)
    theta = pi * (1 + 5**0.5) * indices
    x, y, z = cos(theta) * sin(phi), sin(theta) * sin(phi), cos(phi)
    sphs = []
    for i in range(len(x)):
        spp = icosatiler.crt2SphPol((x[i],y[i],z[i]))
        spp = (rad,spp[1],spp[2])
        sphs.append(spp)
    return sphs
    

In [543]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [544]:
def norm(v):
    m = mag(v)
    vN = [float(i)/float(m) for i in v]
    return vN

In [545]:
def crossProd(u,v):
    return ((u[1]*v[2]-u[2]*v[1]),(u[2]*v[0]-u[0]*v[2]),(u[0]*v[1]-u[1]*v[0]))

In [546]:
def makeParticleModelFromPhenome(particle):
    points = []
    plot = k3d.plot()
    i = 1
    maxEps = 0
    for l in particle.ligands:
        if l.eps>maxEps:
            maxEps = l.eps
    for l in particle.ligands:
        if l.eps > 0.0:
            lx = l.rad*np.sin(l.polAng)*np.cos(l.aziAng)
            ly = l.rad*np.sin(l.polAng)*np.sin(l.aziAng)
            lz = l.rad*np.cos(l.polAng)
            lc = '0x' + ('#%02x%02x%02x' % (0, int((l.eps/maxEps)*255.0), 0))[1:]
            
            plot += k3d.points([lx,ly,lz],point_size=1,color=int(lc, 16))
            #plot += k3d.points([lx,ly,lz],point_size=1,color=int(0x00ffff*(l.eps/maxEps)))
            plot += k3d.text(str(i),[lx,ly,lz],color=0xffffff)
            i+=1
    plot += k3d.points([0,0,0],point_size=7,color=0xff0000)
    return plot

In [547]:
def buildCoveredParticle(eps,rad):
    particle = NanoParticle()
    lPos = icosatiler.cover72SpherePolar(rad)
    for v in lPos:
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [548]:
def spaceIsOccupied(particle,v):
    for l in particle.ligands:
        d = atools.greatArcDist((l.polAng,l.aziAng),(v[1],v[2]),particle.sig)
        if abs(d) < l.size:
            return True
    return False

In [549]:
def getClosestIcos(particle,v):
    lPos = icosatiler.cover72SpherePolar(particle.sig)
    closest = v
    minD = 1e8
    for l in lPos:
        v1 = icosatiler.sphPol2Crt(l) 
        v2 = icosatiler.sphPol2Crt(v)
        d = np.sum([j*j for j in np.subtract(v1,v2)])
        if abs(d) < minD:
            minD = d
            closest = l
    return closest

In [550]:
# lPos = icosatiler.cover72SpherePolar(4)
# dists = []
# minD = 1e8
# for v in lPos:
#     for l in lPos:
#         if l != v:
#             v1 = icosatiler.sphPol2Crt(l) 
#             v2 = icosatiler.sphPol2Crt(v)
#             d = np.sum([j*j for j in np.subtract(v1,v2)])
#             if abs(d) < minD:
#                 minD = d
#     dists.append(np.sqrt(minD))
# print np.mean(dists)

In [551]:
def addLigand(particle,eps,v,lock=False):
    if lock:
        v = getClosestIcos(particle,v)
    if not spaceIsOccupied(particle,v):
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [656]:
def addBelt(particle,eps,rad,n,phi,lock=False):
    step = np.pi*2/float(n)
    for i in range(n):
        v = (rad,step*i,phi)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [553]:
def getArcAngle(dist,rad):
    return dist/rad

In [554]:
def addRing(particle,eps,rad,pos,n,spacing,lock=False):
    step = np.pi*2/float(n)
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    for i in range(n):
        tD = getArcAngle(spacing,rad)
        tP = step*i
        v = (rad,tD,tP)
        vC = icosatiler.sphPol2Crt(v)
        v1 = icosatiler.sphPol2Crt((rad,0,0))
        v2 = icosatiler.sphPol2Crt((rad,pos[0],pos[1]))
        v3 = crossProd(v1,v2)
        if np.sum([j*j for j in v3])== 0.0:
            #it's parallel!
            particle = addLigand(particle,eps,v,lock)
            continue
        dp = np.dot(v1,v2)
        #the magic sauce :)
        ang = np.arctan2(mag(np.cross(v1,v2)), np.dot(v1,v2))
        rM = vectools.buildERMatrix(v3, ang)
        vC = np.dot(rM,vC)
        v = icosatiler.crt2SphPol(vC)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [555]:
def addPatch(particle,eps,rad,pos,n,spacing,lock=False):
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    particle = addLigand(particle,eps,(rad,pos[0],pos[1]),lock)
    particle = addRing(particle,eps,rad,pos,n,spacing,lock)
    return particle

In [556]:
def addMesh(particle,eps,rad,n,lock=False):
    mesh = goldenSpiral(n,particle.sig)
    for point in mesh:
        particle = addLigand(particle,eps,point,lock)
    return particle

In [557]:
#makeParticleModelFromPhenome(buildCoveredParticle(6,4))

In [702]:
zoo = {}
minEps = 160
maxEps = 300
epsStep = 40

In [703]:
for totalEps in xrange(minEps,maxEps,epsStep):
    for i in xrange(24,64,6):
        p = NanoParticle()
        p = addMesh(p,1,4,i,False)
        eps = float(totalEps)/(float(len(p.ligands)))
        for l in p.ligands:
            l.eps = eps
        zoo["sp_"+str(i)+"_"+str(totalEps)] = p

In [704]:
sp = 1.5
for totalEps in xrange(minEps,maxEps,epsStep):
    for n in range(3,8):
        p = NanoParticle()
        for i in range(4):
            p = addPatch(p,1,4,(float(i)*(np.pi*0.5),0),n,sp)
        for i in range(2):
            p = addPatch(p,1,4,((np.pi*0.5),float(i)*(np.pi)+0.5*np.pi),n,sp)
        eps = float(totalEps)/(float(len(p.ligands)))
        for l in p.ligands:
            l.eps = eps
        zoo["cube_"+str(n)+"_"+str(totalEps)] = p

In [705]:
sp = 2.5
for totalEps in xrange(minEps,maxEps,epsStep):
    for n in range(3,8):
        p = NanoParticle()
        for i in range(4):
            p = addRing(p,1,4,(float(i)*(np.pi*0.5),0),n,sp)
        for i in range(2):
            p = addRing(p,1,4,((np.pi*0.5),float(i)*(np.pi)+0.5*np.pi),n,sp)
        eps = float(totalEps)/(float(len(p.ligands)))
        for l in p.ligands:
            l.eps = eps
        zoo["ring_"+str(n)+"_"+str(totalEps)] = p

In [706]:
for totalEps in xrange(minEps,maxEps,epsStep):
    for n in xrange(8,20,2):
        p = NanoParticle()
        p = addBelt(p,1,4,n,0)
        p = addBelt(p,1,4,n,np.pi*0.5)
        eps = float(totalEps)/(float(len(p.ligands)))
        for l in p.ligands:
            l.eps = eps
        zoo["cross_"+str(n)+"_"+str(totalEps)] = p

In [707]:
for totalEps in xrange(minEps,maxEps,epsStep):
    for n in xrange(8,24,4):
        p = NanoParticle()
        p = addBelt(p,1,4,n,0)
        p = addBelt(p,1,4,n,np.pi*0.5)
        p = addRing(p,1,4,(0,0),n,np.pi*2)
        eps = float(totalEps)/(float(len(p.ligands)))
        for l in p.ligands:
            l.eps = eps
        zoo["octet_"+str(n)+"_"+str(totalEps)] = p

In [708]:
makeParticleModelFromPhenome(p)

Plot(antialias=True, background_color=16777215, grid=[-1, -1, -1, 1, 1, 1], height=512, object_ids=[121050363792, 121056787536, 121056904464, 121056423504, 121050363216, 121037633424, 121050363856, 121050364432, 121050364112, 121050362640, 121050363408, 120990354576, 121038401680, 121050363280, 121038401808, 121050362320, 120989917136, 120989914768, 120989916048, 120989914896, 121050362576, 120968717328, 120989918032, 121058015184, 120968717392, 121050363600, 121038401616, 121058013264, 121050364560, 120989913872, 120989914128, 120992394896, 120995496144, 121058016080, 121038074064, 121058015504, 121038074384, 121058015248, 121057107024, 121057107408, 120992395408, 121038074512, 121057106384, 121057107664, 121057107536, 120992395344, 120992395472, 121057106704, 121038970320, 121038970448, 121038714704, 121056323280, 121038714320, 121057107152, 121038715024, 121057107728, 121055296656, 121038714640, 121055295952, 121038714384, 121055295824, 121038715792, 121039851344, 121055296080, 121055297296, 121055296272, 121055296528, 121039851408, 121034514128, 121030234448, 121030234576, 121055294800, 121034514000, 121030234192, 121030234704, 121030235792, 121030236112, 121030234832, 121030236368, 121030235280, 121030236560, 121030235920, 121030235984, 121030236944, 121030235216, 121030236624, 121030237328, 121030238160, 121030235472, 121030236880, 121030237200, 121030235088, 121030235536, 121044439440, 121044439824, 121030237840, 121044440656, 121030238096, 121044440528, 121044440912, 121044439952, 121044441552, 121044440592, 121044440336, 121044440720, 121044441104, 121044441680, 121044441424, 121050364816])

In [718]:
with open('zoo.pickle', 'wb') as handle:
    pickle.dump(zoo, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [709]:
RUNTIME = 25000
TIMESTEP = 0.01
OUTDIR = "/Users/joelforster/Projects/optidb/models/out/"
RUNDIR = "/Users/joelforster/Projects/optidb/models/run/"
TEMPLATEDIR = "/Users/joelforster/Projects/optihedron/mem/template"
TEMPLATEDATAPATH = os.path.join(TEMPLATEDIR,'data.template')
TEMPLATEINPUTPATH = os.path.join(TEMPLATEDIR,'in.template')

ensure_dir(OUTDIR)
ensure_dir(RUNDIR)

for k,v in zoo.iteritems():

    sim = MembraneSimulation(
            k,
            v,
            RUNTIME,
            TIMESTEP,        
            OUTDIR,
            RUNDIR,
            TEMPLATEDATAPATH,
            TEMPLATEINPUTPATH,
            rAxis=vectools.randomUnitVector(),
            rAmount=random.uniform(0.3141,3.141)        
            )
    sim.saveFiles()

saved files: /Users/joelforster/Projects/optidb/models/run/ring_3_200_data.data, /Users/joelforster/Projects/optidb/models/run/ring_3_200_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/sp_48_240_data.data, /Users/joelforster/Projects/optidb/models/run/sp_48_240_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/ring_5_200_data.data, /Users/joelforster/Projects/optidb/models/run/ring_5_200_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/ring_5_160_data.data, /Users/joelforster/Projects/optidb/models/run/ring_5_160_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/octet_12_160_data.data, /Users/joelforster/Projects/optidb/models/run/octet_12_160_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/ring_3_280_data.data, /Users/joelforster/Projects/optidb/models/run/ring_3_280_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/sp_54_200_data.data, /Users/joelforster/Projects/op

saved files: /Users/joelforster/Projects/optidb/models/run/cross_18_200_data.data, /Users/joelforster/Projects/optidb/models/run/cross_18_200_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/cross_8_160_data.data, /Users/joelforster/Projects/optidb/models/run/cross_8_160_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/cross_14_280_data.data, /Users/joelforster/Projects/optidb/models/run/cross_14_280_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/sp_60_240_data.data, /Users/joelforster/Projects/optidb/models/run/sp_60_240_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/octet_16_160_data.data, /Users/joelforster/Projects/optidb/models/run/octet_16_160_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/cube_7_160_data.data, /Users/joelforster/Projects/optidb/models/run/cube_7_160_script.in
saved files: /Users/joelforster/Projects/optidb/models/run/cube_4_280_data.data, /Users/joelforster/

[(4, 0.7227342478134157, -1.199981614864327),
 (4, 1.318116071652818, 2.6832404625866064),
 (4, 1.8234765819369751, 0.2832772328579506),
 (4, 2.4188584057763776, -2.1166859968707015)]